# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Spark: Structured Streaming (QueryListener & Output sinks)** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Files") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

### Creación del FileStream

In [ ]:
log_lines = spark \
                .readStream \
                .format("text") \
                .option("maxFilesPerTrigger", 1) \
                .load("/home/jovyan/notebooks/data/log_streaming/input/")

### Extract traffic info

In [ ]:
from pyspark.sql.functions import split

log_df = log_lines.select(split(log_lines.value, " | ").alias("logs_array"))
log_df = log_df.withColumn("timestamp", log_df["logs_array"].getItem(0).cast("string"))
log_df = log_df.withColumn("level", log_df["logs_array"].getItem(1).cast("string"))
log_df = log_df.withColumn("message", log_df["logs_array"].getItem(2).cast("string"))
log_df = log_df.withColumn("server", log_df["logs_array"].getItem(3).cast("string"))

log_df.printSchema()

### Extract vehicles that exceed 80 KM/H

In [ ]:
log_df = log_df.filter(log_df.level == "WARN")

### Configuración del "Sink" del stream

In [ ]:
query = log_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='2 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(10)

In [ ]:
query_files = log_df \
                .writeStream \
                .format("parquet") \
                .option("path", "/home/jovyan/notebooks/data/log_streaming/output/") \
                .option("checkpointLocation", "/home/jovyan/checkpoint") \
                .start()
query_files.awaitTermination(10)

### Verify data

In [ ]:
speed_df = spark \
            .read \
            .parquet("/home/jovyan/notebooks/data/log_streaming/output/")

speed_df.show(4, False)

In [79]:
sc.stop()